In [1]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import config
import torch

/home/ubuntu/miniconda3/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained(config.model_id)
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids("<|finetune_right_pad_id|>")

In [3]:
model = AutoModelForCausalLM.from_pretrained(config.model_id, 
                                             quantization_config=config.bnb_config,  
                                             device_map='auto', 
                                             torch_dtype=torch.bfloat16)
model.config.pad_token_id = tokenizer.pad_token_id  

lora_model = PeftModel.from_pretrained(model, "model1")

Loading checkpoint shards: 100%|██████████| 4/4 [02:51<00:00, 42.84s/it]


In [4]:
prompt = ["One wise man said:",
          "One wise man said:",
          "One wise man said: when you work",
          "One wise man said: when you're dreaming of something",
          "One wise man said: the meaning of life in",
          "One wise man said: beautiful women",
          "One wise man said: the wisdom lies in",
          "One wise man said: happiness lies in",
          "One wise man said: friends",
          "One wise man said: death",
          "One wise man said: fear",
          ]


In [5]:
inputs = tokenizer(prompt, return_tensors='pt', padding=True).to('cuda')
outputs = lora_model.generate(**inputs, do_sample = True, top_p = 0.8, max_new_tokens=64, num_return_sequences=3)
generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for gen in generated:
    print(gen)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


One wise man said: your love is a fire, and i am a moth.
One wise man said: the love of a good woman is the best thing in the world.
One wise man said: the truth is, you are the only one who can make you happy.
One wise man said: the best way to love someone is to show them how to love themselves.
One wise man said: the world is not a beautiful place, but the people who live in it are.
One wise man said: the world is a mirror of your thoughts.
One wise man said: when you work the hardest, you will find the best.
One wise man said: when you work the whole day and still feel you are not working, you are not working.
One wise man said: when you work the best you can, you will be the best.
One wise man said: when you're dreaming of something, it's not a dream anymore.
One wise man said: when you're dreaming of something, you're dreaming of something that doesn't exist.
One wise man said: when you're dreaming of something, you're dreaming of something you don't have, which means you're not 

## Очень даже красиво!!
Мне понравились эти:
- the truth is, you are the only one who can make you happy
- the world is not a beautiful place, but the people who live in it are
- the world is a mirror of your thoughts
- when you work the hardest, you will find the best
- the wisdom lies in is the ability to recognize the truth
- happiness lies in the joy of achievement (вот это прямо 100/100)
- fear the fear of failure, not failure itself
- death the last of life's great adventures